# Movie Recommendation using Knowledge Graphs

# Installation Of Modules

In [1]:
pip install --upgrade py2neo

In [2]:
 pip install --upgrade --pre py2neo

Note: you may need to restart the kernel to use updated packages.


In [3]:
from py2neo import Graph,Node,Relationship

In [4]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "movie"))

In [5]:
pip install neo4jupyter

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install neo4j
from neo4j import GraphDatabase
import neo4jupyter
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "movie"))


In [7]:
neo4jupyter.init_notebook_mode()

<IPython.core.display.Javascript object>

In [8]:
query="""LOAD CSV WITH HEADERS FROM 'file:///movies.csv' AS line
 MERGE (m:Movie{ id:line.movieId, title:line.title}) 
 FOREACH (gName in split(line.genres, '|') | 
    MERGE (g:Genre {name:gName}) 
    MERGE (m)-[:IS_GENRE]->(g)
)"""
graph.run(query).data()

[]

In [9]:
query="""Match (n:Movie) return n.id as MovieId , n.title as title """
mdf=graph.run(query).to_data_frame()
mdf.head()

,MovieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [10]:
query="""MATCH p=()-[r:IS_GENRE]->() RETURN p LIMIT 25"""
mdf=graph.run(query).data()
mdf[0:5]

[{'p': Path(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Adventure')))},
 {'p': Path(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Animation')))},
 {'p': Path(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Children')))},
 {'p': Path(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Comedy')))},
 {'p': Path(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Fantasy')))}]

In [11]:
query="""LOAD CSV WITH HEADERS FROM "file:///ratings.csv" AS line
 MATCH (m:Movie {id:line.movieId})
 MERGE (u:User {id:line.userId})
 MERGE (u)-[:RATED { rating: toFloat(line.rating)}]->(m);"""
graph.run(query).data()

[]

In [12]:
query="""Match (n:User) return n.id as UserId """
mdf=graph.run(query).to_data_frame()
print(mdf.head())
print("No of users are " + str(mdf.size))

  UserId
0      1
1      2
2      3
3      4
4      5
No of users are 610


In [13]:
query="""MATCH p=()-[r:RATED]->() RETURN p LIMIT 25"""
mdf=graph.run(query).data()
mdf[0:5]

[{'p': Path(Node('User', id='1'), RATED(Node('User', id='1'), Node('Movie', id='661', title='James and the Giant Peach (1996)', tmdbId='10539'), rating=5.0))},
 {'p': Path(Node('User', id='1'), RATED(Node('User', id='1'), Node('Movie', id='3034', title='Robin Hood (1973)', tmdbId='11886'), rating=5.0))},
 {'p': Path(Node('User', id='1'), RATED(Node('User', id='1'), Node('Movie', id='1473', title='Best Men (1997)', tmdbId='19952'), rating=4.0))},
 {'p': Path(Node('User', id='1'), RATED(Node('User', id='1'), Node('Movie', id='3273', title='Scream 3 (2000)', tmdbId='4234'), rating=5.0))},
 {'p': Path(Node('User', id='1'), RATED(Node('User', id='1'), Node('Movie', id='2387', title='Very Bad Things (1998)', tmdbId='10029'), rating=5.0))}]

In [14]:
query="""LOAD CSV WITH HEADERS FROM "file:///tags.csv" AS line
 MATCH (m:Movie {id:line.movieId})
 MATCH (u:User {id:line.userId})
 CREATE (u)-[:TAGGED { tag: line.tag}]->(m);"""
graph.run(query).data()

[]

In [15]:
query="""MATCH p=()-[r:ACTED_IN]->() RETURN p LIMIT 25 """
mdf=graph.run(query).to_data_frame()
mdf.head()

,p
0,({'role': 'Richard Weidner'})
1,({'role': 'Paulie Romano'})
2,({'role': 'Caller #1 - Bob'})
3,({'role': 'Joey Randone'})
4,({'role': 'Wirey Pink'})


In [16]:
query="""MATCH p=()-[r:TAGGED]->() RETURN p LIMIT 25"""
mdf=graph.run(query).data()
mdf[0:5]

[{'p': Path(Node('User', id='2'), TAGGED(Node('User', id='2'), Node('Movie', id='89774', title='Warrior (2011)', tmdbId='59440'), tag='Boxing story'))},
 {'p': Path(Node('User', id='2'), TAGGED(Node('User', id='2'), Node('Movie', id='60756', title='Step Brothers (2008)', tmdbId='12133'), tag='will ferrell'))},
 {'p': Path(Node('User', id='2'), TAGGED(Node('User', id='2'), Node('Movie', id='89774', title='Warrior (2011)', tmdbId='59440'), tag='MMA'))},
 {'p': Path(Node('User', id='2'), TAGGED(Node('User', id='2'), Node('Movie', id='60756', title='Step Brothers (2008)', tmdbId='12133'), tag='Highly quotable'))},
 {'p': Path(Node('User', id='2'), TAGGED(Node('User', id='2'), Node('Movie', id='106782', title='Wolf of Wall Street, The (2013)', tmdbId='106646'), tag='Leonardo DiCaprio'))}]

In [17]:
query="""LOAD CSV WITH HEADERS FROM "file:///links.csv" AS line
MATCH (m:Movie {id:line.movieId})
SET m.tmdbId=line.tmdbId;"""
graph.run(query).data()

[]

In [18]:
query="""LOAD CSV WITH HEADERS FROM "file:///directors.csv" AS line
MATCH (m:Movie{ tmdbId:line.movieId})
MERGE (p:Person{name:line.person_name})
MERGE (p)-[:DIRECTED]->(m);"""
graph.run(query).data()

[]

In [19]:
query="""LOAD CSV WITH HEADERS FROM "file:///roles.csv" AS line
MATCH (m:Movie{ tmdbId:line.movieId})
MERGE (p:Person{name:line.person_name})
CREATE (p)-[r:ACTED_IN] ->(m)
SET  r.role= line.role;"""
graph.run(query).data()

[]

In [20]:
print("Knowledge graph")
(neo4jupyter.draw(graph,{'Movie':'title','User':'id','Person':'name','Genre':'name'}))

Knowledge graph


In [21]:
query="""MATCH (m:Movie {title: "Toy Story (1995)"})-[:ACTED_IN|:IS_GENRE|:DIRECTED]-(p)
RETURN m.id as Movieid,m.title as title,m.tmdbId as tmdbId, p.name as Genre_Crew_Actor limit 10"""
graph.run(query).to_data_frame()


,Movieid,title,tmdbId,Genre_Crew_Actor
0,1,Toy Story (1995),862,Adventure
1,1,Toy Story (1995),862,Animation
2,1,Toy Story (1995),862,Children
3,1,Toy Story (1995),862,Comedy
4,1,Toy Story (1995),862,Fantasy
5,1,Toy Story (1995),862,John Lasseter
6,1,Toy Story (1995),862,John Ratzenberger
7,1,Toy Story (1995),862,R. Lee Ermey
8,1,Toy Story (1995),862,Laurie Metcalf
9,1,Toy Story (1995),862,Tim Allen


In [22]:
query="""MATCH (m:Movie {title: "Toy Story (1995)"})-[:RATED|:TAGGED]-(u)
RETURN m.id as Movieid,m.title as title,m.tmdbId as tmdbId, u.id as Userid LIMIT 10"""
graph.run(query).to_data_frame()

,Movieid,title,tmdbId,Userid
0,1,Toy Story (1995),862,298
1,1,Toy Story (1995),862,161
2,1,Toy Story (1995),862,276
3,1,Toy Story (1995),862,332
4,1,Toy Story (1995),862,399
5,1,Toy Story (1995),862,559
6,1,Toy Story (1995),862,82
7,1,Toy Story (1995),862,223
8,1,Toy Story (1995),862,185
9,1,Toy Story (1995),862,247


In [23]:
user_id = '220'

In [24]:
query="""MATCH (user:User{id:$userid})-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other:User)-[r3:RATED]->(m2:Movie)
WHERE r1.rating > 3 AND r2.rating > 3 AND r3.rating > 3 AND NOT (user)-[:RATED]->(m2)
RETURN distinct m2.id as Movieid,m2.title as title,m2.tmdbId as tmdbId, count(*) AS score
ORDER BY score DESC
LIMIT 15"""
graph.run(query,userid = user_id).to_data_frame()

,Movieid,title,tmdbId,score
0,593,"Silence of the Lambs, The (1991)",274,7203
1,4993,"Lord of the Rings: The Fellowship of the Ring,...",120,6563
2,2858,American Beauty (1999),14,6227
3,110,Braveheart (1995),197,5894
4,3578,Gladiator (2000),98,5777
5,527,Schindler's List (1993),424,5663
6,1136,Monty Python and the Holy Grail (1975),762,5377
7,4963,Ocean's Eleven (2001),161,5011
8,1214,Alien (1979),348,4951
9,1682,"Truman Show, The (1998)",37165,4867


In [25]:
query="""MATCH (user:User{id:$userid})-[r:RATED]-(m)
WITH user, avg(r.rating) AS average
MATCH (user)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other:User)-[r3:RATED]->(m2:Movie)
WHERE r1.rating > average AND r2.rating > average AND r3.rating > average AND NOT (user)-[:RATED]->(m2)
RETURN distinct m2.id as Movieid,m2.title as title,m2.tmdbId as tmdbId, count(*) AS score
ORDER BY score DESC
LIMIT 15"""
graph.run(query,userid = user_id).to_data_frame()

,Movieid,title,tmdbId,score
0,593,"Silence of the Lambs, The (1991)",274,5322
1,4993,"Lord of the Rings: The Fellowship of the Ring,...",120,4276
2,2858,American Beauty (1999),14,4129
3,527,Schindler's List (1993),424,4086
4,110,Braveheart (1995),197,3982
5,3578,Gladiator (2000),98,3537
6,1214,Alien (1979),348,3502
7,1136,Monty Python and the Holy Grail (1975),762,3408
8,1221,"Godfather: Part II, The (1974)",240,3330
9,457,"Fugitive, The (1993)",5503,3222


In [26]:
query="""MATCH (user:User{id:$userid})-[r:RATED]-(m)
WITH user, avg(r.rating) AS average
MATCH (user)-[t1:TAGGED]->(m:Movie)-[r:RATED]-(user)
MATCH (other:User)-[t2:TAGGED]->(m1:Movie)
WHERE r.rating > average AND t1.tag=t2.tag AND  NOT (user)-[:TAGGED]->(m1) AND  NOT (user)-[:RATED]->(m1)
RETURN m1.id as Movieid,m1.title as title,m1.tmdbId as tmdbId, other.id as OtherUserid"""
graph.run(query,userid = user_id).to_data_frame()

""


In [27]:
query="""MATCH (user:User{id:$userid})-[r:RATED]-(m:Movie)
WITH user, avg(r.rating) AS average
MATCH (user)-[r:RATED]->(m:Movie)-[:ACTED_IN]-(p:Person)
WHERE r.rating > average
RETURN  p.name as actor, COUNT(*) AS score 
ORDER BY score DESC LIMIT 10"""
graph.run(query,userid = user_id).to_data_frame()


,actor,score
0,Tom Hanks,198
1,John Ratzenberger,198
2,Samuel L. Jackson,180
3,Harrison Ford,144
4,James Earl Jones,126
5,Bill Paxton,126
6,Joe Ranft,126
7,Hugo Weaving,108
8,Bruce Willis,108
9,Wallace Shawn,90


In [28]:
query="""MATCH (user:User{id:$userid})-[r:RATED]-(m:Movie)
WITH user, avg(r.rating) AS average
MATCH (user)-[r:RATED]->(m:Movie)-[:DIRECTED]-(p:Person)
WHERE r.rating > average
RETURN  p.name as director, COUNT(*) AS score 
ORDER BY score DESC LIMIT 10"""
graph.run(query,userid = user_id).to_data_frame()

,director,score
0,Steven Spielberg,8
1,Robert Zemeckis,6
2,James Cameron,5
3,Joel Coen,3
4,Andrew Stanton,3
5,George Lucas,3
6,Lilly Wachowski,3
7,Quentin Tarantino,3
8,John Lasseter,3
9,Sam Raimi,3


In [29]:
query="""MATCH (user:User{id:$userid})-[r:RATED]-(m:Movie)
WITH user, avg(r.rating) AS average
MATCH (user)-[r:RATED]->(m:Movie)-[:IS_GENRE]-(p:Genre)
WHERE r.rating > average
RETURN  p.name as genre, COUNT(*) AS score 
ORDER BY score DESC LIMIT 10"""
graph.run(query,userid = user_id).to_data_frame()

,genre,score
0,Action,63
1,Adventure,59
2,Comedy,49
3,Drama,43
4,Thriller,40
5,Sci-Fi,39
6,Crime,27
7,Children,19
8,Animation,18
9,Fantasy,18


In [30]:
query="""MATCH (user:User{id:$userid})-[r:RATED]-(m:Movie)
WITH user, avg(r.rating) AS average
MATCH (user)-[r:RATED]->(m:Movie)
WHERE r.rating > average
MATCH (m)-[:IS_GENRE]->(g:Genre)<-[:IS_GENRE]-(rm:Movie)
WITH user, m, rm, COUNT(*) AS gs
OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Person)-[:ACTED_IN]->(rm)
WITH user, m, rm, gs, COUNT(a) AS as
OPTIONAL MATCH (m)<-[:DIRECTED]-(d:Person)-[:DIRECTED]->(rm)
WITH user, m, rm, gs, as, COUNT(d) AS ds
MATCH (rm)
WHERE  NOT (user)-[:RATED]->(rm)
RETURN rm.id as MovieId,rm.title AS title,
(4*gs)+(3*as)+(3*ds) AS weighed_score 
ORDER BY weighed_score DESC LIMIT 10"""
graph.run(query,userid = user_id).to_data_frame()

,MovieId,title,weighed_score
0,54001,Harry Potter and the Order of the Phoenix (2007),26256
1,54001,Harry Potter and the Order of the Phoenix (2007),24312
2,69844,Harry Potter and the Half-Blood Prince (2009),22368
3,54001,Harry Potter and the Order of the Phoenix (2007),22364
4,69844,Harry Potter and the Half-Blood Prince (2009),20424
5,54001,Harry Potter and the Order of the Phoenix (2007),20420
6,69844,Harry Potter and the Half-Blood Prince (2009),20420
7,69844,Harry Potter and the Half-Blood Prince (2009),18476
8,53125,Pirates of the Caribbean: At World's End (2007),16539
9,52722,Spider-Man 3 (2007),13627


In [31]:
query="""MATCH (user:User{id:$userid})-[r:RATED]-(m:Movie)
WITH user, avg(r.rating) AS average
MATCH (user)-[r:RATED]->(m:Movie)
WHERE r.rating > 4.5
MATCH (m)<-[:ACTED_IN]-(a:Person)-[:ACTED_IN]->(rm)
MATCH (rm)
WHERE  NOT (user)-[:RATED]->(rm)
RETURN rm.id as MovieId,rm.title as title,rm.tmdbId as tmdbId, a.name as Actor_name, user.id as userId ,m.id as Movieid,m.title as Movietitle,m.tmdbId as tmdbid LIMIT 10"""
graph.run(query,userid = user_id).to_data_frame()

,MovieId,title,tmdbId,Actor_name,userId,Movieid,Movietitle,tmdbid
0,111659,Maleficent (2014),102651,João Costa Menezes,220,48774,Children of Men (2006),9693
1,53000,28 Weeks Later (2007),1562,João Costa Menezes,220,48774,Children of Men (2006),9693
2,3578,Gladiator (2000),98,João Costa Menezes,220,48774,Children of Men (2006),9693
3,44191,V for Vendetta (2006),752,João Costa Menezes,220,48774,Children of Men (2006),9693
4,130073,Cinderella (2015),150689,João Costa Menezes,220,48774,Children of Men (2006),9693
5,44759,Basic Instinct 2 (2006),3093,João Costa Menezes,220,48774,Children of Men (2006),9693
6,111659,Maleficent (2014),102651,João Costa Menezes,220,48774,Children of Men (2006),9693
7,44759,Basic Instinct 2 (2006),3093,João Costa Menezes,220,48774,Children of Men (2006),9693
8,53000,28 Weeks Later (2007),1562,João Costa Menezes,220,48774,Children of Men (2006),9693
9,44191,V for Vendetta (2006),752,João Costa Menezes,220,48774,Children of Men (2006),9693


In [32]:
#Jaccard
query="""MATCH (user:User {id:$userid})-[r:RATED]->(m:Movie)
WITH user, avg(r.rating) AS user_average
MATCH (user)-[r1:RATED]-> (m:Movie) <-[r2:RATED]-(other)
WITH user, user_average, other,count(distinct m) as intersection, COLLECT({r1: r1, r2: r2}) AS ratings WHERE size(ratings) > 10
MATCH (other)-[r:RATED]->(m:Movie)
WITH user, user_average, other,m, avg(r.rating) AS other_average, ratings,intersection
UNWIND ratings AS r
WITH count(distinct m) as union,user,other,intersection 
RETURN user.id as UserId, other.id as OtherUid, intersection/(1.0 * union) as jacard_similarity
ORDER BY jacard_similarity DESC"""
jacard_df = graph.run(query,userid = user_id).to_data_frame()
jacard_df

,UserId,OtherUid,jacard_similarity
0,220,557,0.814815
1,220,77,0.724138
2,220,399,0.722222
3,220,439,0.714286
4,220,569,0.700000
...,...,...,...
448,220,603,0.066808
449,220,474,0.065465
450,220,89,0.063707
451,220,599,0.061743


In [33]:
OtherUidList = jacard_df['OtherUid'].tolist()
jacard_Similarity = jacard_df['jacard_similarity'].tolist()

In [34]:
#Pearson correlation coefficient
query="""MATCH (user:User {id:$userid})-[r:RATED]->(m:Movie)
WITH user, avg(r.rating) AS user_average
MATCH (user)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other)
WITH user, user_average, other, COLLECT({r1: r1, r2: r2}) AS ratings WHERE size(ratings) > 10
MATCH (other)-[r:RATED]->(m:Movie)
WITH user, user_average, other, avg(r.rating) AS other_average, ratings
UNWIND ratings AS r
WITH sum( (r.r1.rating - user_average) * (r.r2.rating- other_average) ) AS a,
sqrt( sum( (r.r1.rating - user_average)^2) * sum( (r.r2.rating - other_average) ^2)) AS b,
user, other
WHERE b <> 0
RETURN user.id as UserId, other.id as OtherUid, a/b as correlation
ORDER BY correlation DESC"""
graph.run(query,userid = user_id).to_data_frame()

,UserId,OtherUid,correlation
0,220,494,0.782532
1,220,32,0.781892
2,220,485,0.763311
3,220,97,0.754797
4,220,79,0.739910
...,...,...,...
448,220,37,-0.320023
449,220,492,-0.404396
450,220,416,-0.460879
451,220,395,-0.575922


In [35]:
query="""MATCH (user:User {id:$userid})-[r1:RATED]->(m:Movie)
MATCH (other:User {id:"494"})-[r2:RATED]->(m:Movie)
RETURN user.id as Userid, other.id as otherUid,m.title as title,r1.rating as r1,r2.rating as r2"""
graph.run(query,userid = user_id).to_data_frame()

,Userid,otherUid,title,r1,r2
0,220,494,"Godfather, The (1972)",5.0,5.0
1,220,494,Terminator 2: Judgment Day (1991),5.0,5.0
2,220,494,Aliens (1986),5.0,5.0
3,220,494,Raiders of the Lost Ark (Indiana Jones and the...,5.0,5.0
4,220,494,Star Wars: Episode IV - A New Hope (1977),5.0,5.0
5,220,494,Star Wars: Episode VI - Return of the Jedi (1983),4.5,4.0
6,220,494,Die Hard (1988),5.0,5.0
7,220,494,"Matrix, The (1999)",5.0,5.0
8,220,494,Star Wars: Episode V - The Empire Strikes Back...,5.0,5.0
9,220,494,Indiana Jones and the Last Crusade (1989),5.0,5.0


In [36]:
query="""MATCH (user:User {id:$userid})-[r:RATED]->(m:Movie)
WITH user, avg(r.rating) AS user_average
MATCH (user)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other)
WITH user, user_average, other, COLLECT({r1: r1, r2: r2}) AS ratings WHERE size(ratings) > 10
MATCH (other)-[r:RATED]->(m:Movie)
WITH user, user_average, other, avg(r.rating) AS other_average, ratings
UNWIND ratings AS r
WITH sum( (r.r1.rating- user_average) * (r.r2.rating- other_average) ) AS a,
sqrt( sum( (r.r1.rating - user_average)^2) * sum( (r.r2.rating - other_average) ^2)) AS b,user, other 
WHERE b <> 0
WITH user, other, a/b as correlation
ORDER BY correlation DESC LIMIT 10
MATCH (other)-[r:RATED]->(m:Movie) WHERE NOT EXISTS( (user)-[:RATED]->(m) )
WITH m,  SUM( correlation * r.rating) AS score, COLLECT(other) AS other
RETURN m.id as MovieId,m.title as title,m.tmdbId as tmdbId, other, score
ORDER BY score DESC LIMIT 10"""
pearsondf = graph.run(query,userid = user_id).to_data_frame()
pearsondf

,MovieId,title,tmdbId,other,score
0,593,"Silence of the Lambs, The (1991)",274,"[{'id': '32'}, {'id': '485'}, {'id': '97'}, {'...",20.070192
1,527,Schindler's List (1993),424,"[{'id': '32'}, {'id': '88'}, {'id': '235'}, {'...",13.163388
2,457,"Fugitive, The (1993)",5503,"[{'id': '485'}, {'id': '79'}, {'id': '235'}, {...",12.400923
3,110,Braveheart (1995),197,"[{'id': '494'}, {'id': '485'}, {'id': '124'}, ...",11.583822
4,440,Dave (1993),11566,"[{'id': '32'}, {'id': '235'}, {'id': '436'}, {...",11.450195
5,592,Batman (1989),268,"[{'id': '32'}, {'id': '485'}, {'id': '235'}, {...",10.462137
6,246,Hoop Dreams (1994),14275,"[{'id': '32'}, {'id': '79'}, {'id': '235'}]",9.693276
7,161,Crimson Tide (1995),8963,"[{'id': '485'}, {'id': '235'}, {'id': '436'}]",8.783676
8,539,Sleepless in Seattle (1993),858,"[{'id': '32'}, {'id': '235'}, {'id': '436'}, {...",8.766594
9,3578,Gladiator (2000),98,"[{'id': '494'}, {'id': '97'}, {'id': '79'}]",8.411665


In [37]:
l = pearsondf['other']
score = pearsondf['score']
combineddf = pearsondf
for i in range(len(l)):
    total= 0
    for j in l[i]:
        c = j['id']
        a = OtherUidList.index(c)
        total+=jacard_Similarity[a]
    combineddf.at[i,'score'] = total*score[i]
combineddf = combineddf.sort_values(by='score',ascending=False)
combineddf

,MovieId,title,tmdbId,other,score
0,593,"Silence of the Lambs, The (1991)",274,"[{'id': '32'}, {'id': '485'}, {'id': '97'}, {'...",38.011325
3,110,Braveheart (1995),197,"[{'id': '494'}, {'id': '485'}, {'id': '124'}, ...",21.002466
2,457,"Fugitive, The (1993)",5503,"[{'id': '485'}, {'id': '79'}, {'id': '235'}, {...",16.109217
1,527,Schindler's List (1993),424,"[{'id': '32'}, {'id': '88'}, {'id': '235'}, {'...",13.138625
5,592,Batman (1989),268,"[{'id': '32'}, {'id': '485'}, {'id': '235'}, {...",12.270080
9,3578,Gladiator (2000),98,"[{'id': '494'}, {'id': '97'}, {'id': '79'}]",11.019068
4,440,Dave (1993),11566,"[{'id': '32'}, {'id': '235'}, {'id': '436'}, {...",10.349255
7,161,Crimson Tide (1995),8963,"[{'id': '485'}, {'id': '235'}, {'id': '436'}]",8.665393
6,246,Hoop Dreams (1994),14275,"[{'id': '32'}, {'id': '79'}, {'id': '235'}]",8.115560
8,539,Sleepless in Seattle (1993),858,"[{'id': '32'}, {'id': '235'}, {'id': '436'}, {...",7.923684
